In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# data preprocess
from tifffile import imread
from scipy import ndimage


def normalize(volume):
    """Normalize the volume"""
    volume[volume > 0] = 1
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize"""
    # Set the desired depth
    desired_depth = 32
    desired_width = 32
    desired_height = 32
    # Get current depth
    current_depth = img.shape[0]
    current_width = img.shape[-1]
    current_height = img.shape[1]
    # Compute depth factor
    depth_factor = desired_depth / current_depth
    width_factor = desired_width / current_width
    height_factor = desired_height / current_height
    resize_factor = np.amin([depth_factor, width_factor, height_factor])
    # Resize
    img = ndimage.zoom(img, resize_factor, order=0)
    new_depth = round(current_depth * resize_factor)
    new_width = round(current_width * resize_factor)
    new_height = round(current_height * resize_factor)
    a = round((40 - new_depth) / 2)
    b = 40 - new_depth - a
    c = round((40 - new_width) / 2)
    d = 40 - new_width - c
    e = round((40 - new_height) / 2)
    f = 40 - new_height -e
    img = np.pad(img, ((a,b),(e,f),(c,d)))
    img = np.asarray(img).astype('float32')
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = imread(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [3]:
# load data

x_filenames = []
folder = 'Data/ShapeIdentify/Predict/ShewanellaBiofilm'

for filename in os.listdir(folder): #import input folder
    if filename.endswith('.tif'):
        img = imread(os.path.join(folder, filename))
        if img is not None:
            x_filenames.append(filename)
            
print("Objects to predict: " + str(len(x_filenames)))

Objects to predict: 3208


In [4]:
# Load best weights.
model = keras.models.load_model('models/3d_image_classification1008.h5')
#model.load_weights("3d_image_classification.h5")
thresh = 0.5
Output = {}
Output['RodProbability'] = []
Output['Rod'] = []
Output['Filename'] = []
for filename in x_filenames:
    img =  process_scan(os.path.join(folder, filename))
    prediction = model.predict(np.expand_dims(img, axis=0))[0][0]
    if prediction > thresh:
        rod = 1
    else:
        rod = 0
    Output['RodProbability'].append(prediction)
    Output['Rod'].append(rod)
    Output['Filename'].append(filename)


In [5]:
import pandas as pd
df = pd.DataFrame.from_dict(Output)
df.to_csv('Data/ShapeIdentify/Predict/Output1008.csv',index=False)

In [6]:
np.sum(Output['Rod'])/len(Output['Rod'])

0.8628428927680798